# GA Data Science 16 (DAT16) - Lab6

### Regressions

by Justin Breucop, adapted from Francesco Mosconi & Craig Sakuma

In [ ]:
#usual imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from bokeh.plotting import figure,show,output_notebook
output_notebook()

%matplotlib inline

# Statsmodel APIs
import statsmodels.api as sm

## Linear Regression

Load a subset of the housing data

In [ ]:
data = pd.read_csv('../data/housing-data.csv')

In [ ]:
data.head()

In [ ]:
data_n = pd.DataFrame()
for col in data.columns:
    data_n[col] = data[col].apply(lambda x: float(x)/(10**(len(str(data[col].max()))-1)) )
data_n.head()

The dataset contains 4 columns. Our objective is to build a model that is able to predict the price of a house, using the information contained in the other columns (sqft, bedrooms and age)

In [ ]:
from pandas.tools.plotting import scatter_matrix
scat = scatter_matrix(data, figsize = (8,8))

Run 1. x = sqft, data is raw (so graph units are nice right)

Let''s start with a simple model that uses the house surface to estimate the price

In [ ]:
x = data['sqft'].values
y = data['price'].values

In [ ]:
results = sm.OLS(y, x).fit()
slope = results.params[0]
r2 = results.rsquared
print slope
print r2

A little math terminology refresher: slope is the steepness of the line and the r squared value is a "goodness of fit" measure. It tells how much of the variability of our y & x is explained by our model. 0 is bad, 95% is good and 100% is suspect (always be wary of perfect scores).

In [ ]:
# Here we'll build the chart for the graph

p = figure(title='Loop')
p.circle(x,y,color="blue",size=8)

x1 = range(min(x),max(x)+1)

# For loop:
y1 = []
for val in x1:
    y1.append(slope*val)

p.line(x1,y1,color='red')

show(p)

In [ ]:
results.summary()

But we don't have a y intercept, which means the line passes through 0,0 as a requirement. This works for certain models but maybe not for others. 

`sm.add_constant` adds a column of ones to allow for finding an intercept to your data.

In [ ]:
X = sm.add_constant(x, prepend=True)

In [ ]:
results = sm.OLS(y, X).fit()
intercept, slope = results.params

r2 = results.rsquared

print intercept,slope
print r2

In [ ]:

p = figure(title='Price vs Sqft')
p.circle(x,y,color="blue",size=8)

x1 = range(min(x),max(x)+1)

# For loop:
y1 = []
for val in x1:
    y1.append(intercept + slope*val)
# This generates y1 again, but with one line of code instead
# List Comprehension:
y1 = [intercept + slope*val for val in x1]

p.line(x1,y1,color='red')

show(p)

In [ ]:
results.summary()

### EXERCISE #1
#### Create the linear model for data_n to predict price based on age. 
#### What are the slope, intercept and r-squared?

#### Plot the data along with the line for the regression model

#### Bonus challenge: Change the color and shape of the data points.

Confidence Intervals

Something to entertain you, definitely not necessary but fun to know a short cut.

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std

prstd, iv_l, iv_u = wls_prediction_std(results)

In [ ]:

p2 = figure(title="Confidence Intervals for Price to Sqft")
p2.circle(x, y,size=4)
p2.line(x, results.fittedvalues, color='red',line_width=4)
p2.line(x, iv_u, color='red',line_width=2)
p2.line(x, iv_l, color='red',line_width=2)

show(p2)

## Function of Bedrooms?

Run 3:  x = bdrms, data is raw (so graph units are nice)

In [ ]:
x = data['bdrms'].values
y = data['price'].values

X = sm.add_constant(x, prepend=True)
results = sm.OLS(y, X).fit()
intercept, slope = results.params

r2 = results.rsquared

p = figure(title='Price vs Bedrooms')
p.circle(x,y,color="blue",size=8)

x1 = range(min(x),max(x)+1)

# For loop:
y1 = []
for val in x1:
    y1.append(intercept + slope*val)
    
# This generates y1 again, but with one line of code instead
# List Comprehension:
y1 = [intercept + slope*val for val in x1]

p.line(x1,y1,color='red')

show(p)

In [ ]:
results.summary()

In [ ]:
x = data_n['bdrms'].values
y = data_n['price'].values

X = sm.add_constant(x, prepend=True)
results = sm.OLS(y, X).fit()
intercept, slope = results.params
r2 = results.rsquared

p = figure(title='Price vs bdrms')
p.circle(x,y,color="blue",size=8)

x1 = range(int(min(x)),int(max(x)+1))
# For loop:
y1 = []
for val in x1:
    y1.append(intercept + slope*val)
# This generates y1 again, but with one line of code instead
# List Comprehension:
y1 = [intercept + slope*val for val in x1]

p.line(x1,y1,color='red')
show(p)

In [ ]:
print results.summary()

In [ ]:
print results.summary()

## Function of multiple variables

In [ ]:
x = data_n[['sqft', 'bdrms','age']].values
y = data_n['price'].values

X = sm.add_constant(x, prepend=True)
results = sm.OLS(y, X).fit()

In [ ]:
results.summary()

## Just sqft & bedrooms

In [ ]:
x = data_n[['sqft', 'bdrms']].values
y = data_n['price'].values
X = sm.add_constant(x, prepend=True)
results = sm.OLS(y, X).fit()
results.summary()

## The formula API

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
results = smf.ols('price ~ sqft + bdrms', data=data).fit()

In [ ]:
results.summary()

## Nonlinear Fitting

Let's generate points with an arbitrary nonlinear function

$y = \frac{1}{2} x + \frac{1}{2} \sin{x} - \frac{1}{50} (x-5)^2 + 5$

Note: np.c\_ is shorthand in numpy for combining columns

In [ ]:
# start with some fake data
nsample = 50
sig = 0.5
# generate linear space
x = np.linspace(0, 20, nsample)
# invent function
X = np.c_[x, np.sin(x), (x - 5)**2, np.ones(nsample)]
# invent coefficients
beta = [1.5, 2.5, -0.2, 15.]
# generate |
y_true = np.dot(X, beta)
# add noise
y = y_true + sig * np.random.normal(size=nsample)

In [ ]:
X = np.c_[x, np.sin(x), (x - 5)**2, np.ones(nsample)]
X.shape
x.shape
y_true.shape

In [ ]:
np.random.normal(size=nsample)

quick look at the data

In [ ]:
p = figure()
p.line(x, y_true,line_color='blue')
p.circle(x, y, color='green')
show(p)

In [ ]:
res = sm.OLS(y, X).fit()
res.summary()

I can access the results attributes

In [ ]:
print res.params

In [ ]:
print 
res.bse

In [ ]:
print res.predict()

Let's look at the fit!

In [ ]:
p = figure(title='Blue = True; Red = OLS')
p.circle(x, y, color='blue')
p.line(x, y_true, color='blue',line_width = 3)

prstd, iv_l, iv_u = wls_prediction_std(res)

p.line(x, res.fittedvalues, line_color= 'red', line_dash='dashed',line_width = 3)
p.line(x, iv_u, line_color= 'red', line_dash='dashed')
p.line(x, iv_l, line_color= 'red', line_dash='dashed')
show(p)

## Overfitting

In [ ]:
x = np.array([-0.99768, -0.69574, -0.40373, -0.10236, 0.22024, 0.47742, 0.82229,1.20044])
y = np.array([2.0885, 1.1646, 0.3287, 0.46013, 0.44808, 0.10013, -0.32952, -0.32811])

p = figure()
p.circle(x, y, size=8,color='blue')
show(p)

np.vander yields a vandermonde matrix (fancy way of doing columns of x, x^2, x^3 etc. up to the second argument)

In [ ]:
# X = np.c_[x, np.ones(len(x))]
X = np.vander(x, 4)
# Same as np.c_[x]

In [ ]:
X

In [ ]:
res = sm.OLS(y, X).fit()
res.summary()

### Linear fit

In [ ]:
p = figure(title='Blue = True; Red = OLS')
p.circle(x, y, size=8,color='blue')

xx = np.linspace(-1.25,1.25,100)
p.line(xx, res.predict(np.vander(xx,2)), color='red')
show(p)

### Quadratic

In [ ]:
X = np.c_[x**2, x, np.ones(len(x))]
res = sm.OLS(y, X).fit()

p = figure(title='Blue = True; Red = OLS')
p.circle(x, y, size=8,color='blue')

xx = np.linspace(-1.25,1.25,100)
p.line(xx, res.predict(np.vander(xx,3)), color='red')
show(p)

### Quintic

In [ ]:
X = np.c_[x**5, x**4, x**3, x**2, x, np.ones(len(x))]
res = sm.OLS(y, X).fit()

p = figure(title='Blue = True; Red = OLS')
p.circle(x, y, size=8,color='blue')

xx = np.linspace(-1.25,1.25,100)
p.line(xx, res.predict(np.vander(xx,6)), color='red')
show(p)

## Regularization

In [ ]:
from sklearn.linear_model import Ridge, Lasso

In [ ]:
ridge = Ridge(alpha = .2)
ridge.fit(np.vander(x, 6), y)

lasso = Lasso(alpha = 1)
lasso.fit(np.vander(x, 6), y)

p = figure(title='Blue = True; Red = OLS')
p.circle(x, y, size=8,color='blue')

xx = np.linspace(-1.25,1.25,100)
p.line(xx, res.predict(np.vander(xx,6)), color='red')
p.line(xx, ridge.predict(np.vander(xx,6)), color='green')
p.line(xx, lasso.predict(np.vander(xx,6)), color='cyan')
show(p)